必要的package

In [31]:
import tensorflow as tf
import pandas as pd
from numpy import random as nr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder  
import types
from __future__ import print_function

一些常量的定义

In [32]:
img_size=28*28
learning_rate=1e-6
batch_size=500
num_data=42000

读入训练数据

In [33]:
labeled_images = pd.read_csv('./input/train.csv')
#images = labeled_images.iloc[0:5000,1:]
#labels = labeled_images.iloc[0:5000,:1]


In [34]:
images = labeled_images.iloc[:,1:]
labels = labeled_images.iloc[:,:1]

切割数据集

In [35]:
(train_images, test_images,train_labels, test_labels) = train_test_split(images, labels, train_size=0.8, random_state=0)
#将DataFrame转换为matrix

In [36]:
training_images=train_images.as_matrix()
training_labels=train_labels.as_matrix()
testing_images=test_images.as_matrix()
testing_labels=test_labels.as_matrix()

onehot encoding

In [37]:
enc = OneHotEncoder()  
enc.fit(training_labels)  
print("enc.n_values_ is:",enc.n_values_  )
training_labels_onehot=enc.transform(training_labels).toarray()
testing_labels_onehot=enc.transform(testing_labels).toarray()

enc.n_values_ is: [10]


构建图

In [38]:
x=tf.placeholder("float",[None,784])#[none,img_size]表示在tf运行时能输入任意形数量的images,每张image尺寸为img_size
#模型参数，初始值均为零
W = tf.Variable(tf.zeros([784,10]))#权重值
b = tf.Variable(tf.zeros([10]))#偏置量
y = tf.nn.softmax(tf.matmul(x,W) + b)

训练模型

In [39]:
y_ = tf.placeholder("float", [None,10])#正确的标签
cross_entropy = -tf.reduce_sum(y_*tf.log(y+1e-10))#
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
init = tf.global_variables_initializer()

In [40]:
sess = tf.Session()
sess.run(init)
for i in range(1000):
    batch_indices=nr.choice(range(int(num_data*0.5)),batch_size)
    batch_xs=training_images[batch_indices,:]
    batch_ys=training_labels_onehot[batch_indices,:]
    
    _,c,W1,b1,y1,y2=sess.run([train_step,cross_entropy,W,b,y,y_], feed_dict={x: batch_xs, y_: batch_ys})
    if i % 50 == 0:
        print("Epoch:", '%04d' % (i+1), "cost=", \
              "{:.9f}".format(c))
        #print("W:",W1)
        #print("b:",b1)
        #print("y:",y1)
        #print("y_:",y2)

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print(sess.run(accuracy, feed_dict={x: testing_images, y_: testing_labels_onehot}))




Epoch: 0001 cost= 1151.291381836
Epoch: 0051 cost= 2712.802978516
Epoch: 0101 cost= 2210.598632812
Epoch: 0151 cost= 2515.295898438
Epoch: 0201 cost= 1861.976684570
Epoch: 0251 cost= 1381.706298828
Epoch: 0301 cost= 1497.905029297
Epoch: 0351 cost= 1875.455566406
Epoch: 0401 cost= 1559.579589844
Epoch: 0451 cost= 1345.133666992
Epoch: 0501 cost= 1946.775634766
Epoch: 0551 cost= 932.602905273
Epoch: 0601 cost= 730.054626465
Epoch: 0651 cost= 1566.018310547
Epoch: 0701 cost= 625.769287109
Epoch: 0751 cost= 658.143676758
Epoch: 0801 cost= 722.992309570
Epoch: 0851 cost= 436.619812012
Epoch: 0901 cost= 683.665100098
Epoch: 0951 cost= 473.338684082
0.908929
